In [1]:
import json
import collections
from collections import defaultdict
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from collections import Counter
import math
from statistics import mean

import numpy as np
from numpy import dot
from numpy.linalg import norm

import scipy
from scipy.optimize import linear_sum_assignment
import nltk
nltk.download('punkt')
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.tokenize import word_tokenize as tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /Users/anyaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('../sparse-1006.json') as f:
    sparse = json.load(f)
# with open('../dense-74.json') as f:
#     dense = json.load(f)
# with open('../sparse-74.json') as f:
#     sparse_74 = json.load(f)

In [2]:
# def clean(x):
#     words = tokenize(x.lower())
#     wl = [stemmer.stem(w) for w in words if w not in stop_words and (w.islower() or w.isalnum())]
#     return wl
# def counter_to_dict(ct):
#     d={}
#     for word, count in ct:
#         d[word] = count
#     return d

# Extract head NP

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [15]:
def lemma(text):
    doc = nlp(text)
    assert(len(doc)==1)
    return (doc[0]).lemma_

In [8]:
def _get_tree(s):
    doc = nlp(s)
    
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
                [child for child in token.children])
    return '****************'

In [10]:
def get_headNP(s, check=False):
    #hard coded fix typo
    if s.startswith('aa '):
        s=s.replace('aa ', 'a ')
    
    #get tree
    doc = nlp(s)
    if check:
        _get_tree(s)
    
    #single word
    if len(doc)==1:
        return doc[0].text
        
    for token in doc:
        if token.dep_=='ROOT' and token.head.pos_ in ['NOUN', 'INTJ', 'PROPN', 'PRON', 'ADJ', 'ADV']: 
            return token.text
         
        if token.dep_=='ROOT' and token.head.pos_=='VERB':
            if list(token.children)[0].dep_=='prep':
                return token.text
            else:
                return list(token.children)[0].text

        if token.dep_=='ROOT' and token.head.pos_=='ADP':
            return list(token.children)[-1].text

        # hard code "xx can" utterances
        if token.dep_=='ROOT' and token.text=='can':
            return token.text
        
#     print('***', s)
#     print(_get_tree(s))
    return 

In [9]:
# get head NP
'''
whole_to_part_to_count = {'person':
    {'count': 800,
    'parts_dict': 
        {
        'head': 10,
        ...
        }
    }
}
'''
whole_to_part_to_count={}
MANUAL_CHECK_WHOLE=[]
MANUAL_CHECK_PART=[]
for i, (tangram, anns) in enumerate(sparse.items()):
    if i%50==0:
        print(i)
        
    for ann in anns:
        w=get_headNP(ann['whole'])
        if w: # head word for whole-ann
            whole_to_part_to_count.setdefault(w, {'count':0, 'parts_dict':defaultdict(int)})
            whole_to_part_to_count[w]['count']+=1
            
            parts = list(set(ann['piece'].values())) # parts *without duplicates*
            for p_ann in parts:
                p=get_headNP(p_ann)
                if p: # head word for part-ann
                    whole_to_part_to_count[w]['parts_dict'][p]+=1
                else: # cant find head word for part-ann
                    MANUAL_CHECK_PART.append({'whole_head': w, 'part_ann': p_ann}) # add to manual check

        else: # cant find head word for whole-ann
            MANUAL_CHECK_WHOLE.append(ann) # add to manual check
            

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


# add head words manually (run once)

In [12]:
MANUAL_CHECK_WHOLE

[{'whole': "a bird flapping it's wings",
  'piece': {'1': 'body',
   '3': 'body',
   '7': 'wings',
   '4': 'body',
   '2': 'head',
   '5': 'wings',
   '6': 'wings'}},
 {'whole': "a bear on it's back legs",
  'piece': {'7': 'legs',
   '5': 'head',
   '1': 'arms',
   '4': 'mouth',
   '2': 'head',
   '6': 'back',
   '3': 'head'}},
 {'whole': "a seal flapping it's fins",
  'piece': {'2': 'body',
   '5': 'fins',
   '3': 'tail',
   '6': 'body',
   '1': 'head',
   '7': 'fins',
   '4': 'tail'}}]

In [13]:
whole_to_part_to_count['bird']['count']+=1
whole_to_part_to_count['bird']['parts_dict']['body']+=1
whole_to_part_to_count['bird']['parts_dict']['wings']+=1
whole_to_part_to_count['bird']['parts_dict']['head']+=1

whole_to_part_to_count['bear']['count']+=1
whole_to_part_to_count['bear']['parts_dict']['legs']+=1
whole_to_part_to_count['bear']['parts_dict']['head']+=1
whole_to_part_to_count['bear']['parts_dict']['arms']+=1
whole_to_part_to_count['bear']['parts_dict']['mouth']+=1
whole_to_part_to_count['bear']['parts_dict']['back']+=1

whole_to_part_to_count['seal']['count']+=1
whole_to_part_to_count['seal']['parts_dict']['body']+=1
whole_to_part_to_count['seal']['parts_dict']['fins']+=1
whole_to_part_to_count['seal']['parts_dict']['tail']+=1
whole_to_part_to_count['seal']['parts_dict']['fins']+=1
whole_to_part_to_count['seal']['parts_dict']['head']+=1

In [14]:
MANUAL_CHECK_PART

[{'whole_head': 'pyramid',
  'part_ann': 'light show with darker shape in middle (the light is around the marked box)'},
 {'whole_head': 'pyramid',
  'part_ann': 'pyramid (it is hard to tell the border between pyramid and ground at night on the left side)'},
 {'whole_head': 'fountain', 'part_ann': 'where the button you press it'},
 {'whole_head': 'house', 'part_ann': 'ten of hearts'},
 {'whole_head': 'women', 'part_ann': 'leg are inside skirt'},
 {'whole_head': 'train', 'part_ann': 'where the wheels are'},
 {'whole_head': 'number', 'part_ann': 'a 5'},
 {'whole_head': 'rose', 'part_ann': 'petal 1'},
 {'whole_head': 'rose', 'part_ann': 'petal 3'},
 {'whole_head': 'beggar', 'part_ann': 'outstretched are'},
 {'whole_head': 'lever', 'part_ann': 'load (must be more dense than load 1)'}]

In [15]:
def _add(_whole, _part):
    whole_to_part_to_count[_whole]['parts_dict'][_part]=whole_to_part_to_count[_whole]['parts_dict'].get(_part,0)+1

In [16]:
_add('pyramid', 'show')
_add('pyramid','pyramid')
_add('fountain','button')
_add('house','ten')
_add('women','leg')
_add('train','wheels')
_add('number','5')
_add('rose','petal')
_add('rose','petal')
_add('beggar','arm') # typo?
_add('lever','load')

In [19]:
with open('./unstemmed_whole_part_counts.json', 'w') as f:
    json.dump(whole_to_part_to_count, f)

# stem and merge words

In [17]:
def merge_dicts(x, y): # merge and sum 2 dicts
    return {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}

In [28]:
# D={}
# for whole_ann, whole_dict in whole_to_part_to_count.items(): # merge stemmed head words in whole-anns
#     st_whole_ann=stemmer.stem(whole_ann)
#     D.setdefault(st_whole_ann, {'count':0, 'parts_dict':{}})
#     D[st_whole_ann]['count']+=whole_dict['count'] # merge whole-head counts
#     D[st_whole_ann]['parts_dict'] = merge_dicts(D[st_whole_ann]['parts_dict'], whole_dict['parts_dict'])

In [39]:
# stemmed_merged=defaultdict(dict)
# for whole_ann, whole_dict in D.items(): # merge stemmed head words in part-anns
#     stemmed_merged[whole_ann]['count']=whole_dict['count']
#     stemmed_parts_dict=defaultdict(int)
#     for part_ann, c in whole_dict['parts_dict'].items():
#         st_part_ann=stemmer.stem(part_ann)
#         stemmed_parts_dict[st_part_ann]+=c
#     stemmed_merged[whole_ann]['parts_dict']=stemmed_parts_dict

In [43]:
# with open('./stemmed_whole_part_counts.json', 'w') as f:
#     json.dump(stemmed_merged, f)

# lemmatize and merge words

In [14]:
with open('./unstemmed_whole_part_counts.json') as f:
    whole_to_part_to_count=json.load(f)

In [18]:
D={}
for whole_ann, whole_dict in whole_to_part_to_count.items(): # merge stemmed head words in whole-anns
    st_whole_ann=lemma(whole_ann)
    D.setdefault(st_whole_ann, {'count':0, 'parts_dict':{}})
    D[st_whole_ann]['count']+=whole_dict['count'] # merge whole-head counts
    D[st_whole_ann]['parts_dict'] = merge_dicts(D[st_whole_ann]['parts_dict'], whole_dict['parts_dict'])

In [20]:
lemma_merged=defaultdict(dict)
for whole_ann, whole_dict in D.items(): # merge stemmed head words in part-anns
    lemma_merged[whole_ann]['count']=whole_dict['count']
    lemma_parts_dict=defaultdict(int)
    for part_ann, c in whole_dict['parts_dict'].items():
        st_part_ann=lemma(part_ann)
        lemma_parts_dict[st_part_ann]+=c
    lemma_merged[whole_ann]['parts_dict']=lemma_parts_dict

In [22]:
lemma_merged.keys()

dict_keys(['chair', 'snake', 'view', 'counter', 'person', 'machine', 'shell', 'envelope', 'box', 'sushi', 'face', 'turtle', 'man', 'bodybuilder', 'submarine', 'superman', 'bat', 'shade', 'robot', 'dog', 'trophy', 'girl', 'body', 'lamp', 'bird', 'crown', 'table', 'bridge', 'top', 'butterfly', 'hat', 'dragon', 'bathtub', 'cow', 'cat', 'head', 'fish', 'firepit', 'hug', 'tent', 'pyramid', 'house', 'anvil', 'kite', 'camp', 'turtil', 'manger', 'helmet', 'bottle', 'sundae', 'fireplace', 'container', 'bowl', 'chiminea', 'vase', 'exhaust', 'armor', 'duck', 'tiki', 'perfume', 'spotlight', 'dish', 'goblet', 'lady', 'airplane', 'doorway', 'nun', 'mouth', 'game', 'seesaw', 'well', 'lemonade', 'stove', 'chimney', 'tree', 'water', 'place', 'podium', 'altar', 'queen', 'snifter', 'stand', 'back', 'glass', 'magician', 'collar', 'light', 'angel', 'horse', 'llama', 'giraffe', 'goat', 'lama', 'mountain', 'cleaver', 'cup', 'furnace', 'fountain', 'sink', 'hood', 'faucet', 'lock', 'scanner', 'ship', 'basket',

In [23]:
with open('./lemmatized_whole_part_counts.json', 'w') as f:
    json.dump(lemma_merged, f)